In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout , BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
import pandas as pd

In [2]:
train_df=pd.read_csv('./data/sign_mnist_train.csv')
test_df=pd.read_csv('./data/sign_mnist_test.csv')

train_df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179


In [3]:
y_train = train_df['label']
y_test = test_df['label']
del train_df['label']
del test_df['label']

In [4]:
from sklearn.preprocessing import LabelBinarizer

label_binarizer = LabelBinarizer()
y_train = label_binarizer.fit_transform(y_train)
y_test = label_binarizer.fit_transform(y_test)

x_train = train_df.values
x_test = test_df.values

x_train = x_train / 255
x_test = x_test / 255

x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)

In [5]:
x_train.shape

(27455, 28, 28, 1)

In [6]:
datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False, 
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=10,
        zoom_range = 0.1, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=False,
        vertical_flip=False
    )

datagen.fit(x_train)

In [7]:
model = Sequential()
model.add(Conv2D(75 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu' , input_shape = (28,28,1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

model.add(Conv2D(50 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

model.add(Conv2D(25 , (3,3) , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))

model.add(Flatten())
model.add(Dense(units = 512 , activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(units = 24 , activation = 'softmax'))


In [8]:
model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 75)        750       
                                                                 
 batch_normalization (Batch  (None, 28, 28, 75)        300       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 75)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 50)        33800     
                                                                 
 dropout (Dropout)           (None, 14, 14, 50)        0         
                                                                 
 batch_normalization_1 (Bat  (None, 14, 14, 50)        

In [9]:
losscallback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
accallback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

In [10]:
nn = model.fit(datagen.flow(x_train,y_train, batch_size = 128) ,epochs = 50 , validation_data = (x_test, y_test),callbacks=[losscallback, accallback])

Epoch 1/50


215/215 [==============================] - 37s 166ms/step - loss: 1.0685 - accuracy: 0.6622 - val_loss: 4.7666 - val_accuracy: 0.0612
Epoch 2/50
215/215 [==============================] - 36s 169ms/step - loss: 0.2204 - accuracy: 0.9272 - val_loss: 4.1771 - val_accuracy: 0.1857
Epoch 3/50
215/215 [==============================] - 35s 165ms/step - loss: 0.1058 - accuracy: 0.9662 - val_loss: 0.4140 - val_accuracy: 0.8691
Epoch 4/50
215/215 [==============================] - 36s 165ms/step - loss: 0.0707 - accuracy: 0.9764 - val_loss: 0.0467 - val_accuracy: 0.9819
Epoch 5/50
215/215 [==============================] - 38s 175ms/step - loss: 0.0421 - accuracy: 0.9870 - val_loss: 0.0412 - val_accuracy: 0.9879
Epoch 6/50
215/215 [==============================] - 36s 166ms/step - loss: 0.0403 - accuracy: 0.9867 - val_loss: 0.0591 - val_accuracy: 0.9808
Epoch 7/50
215/215 [==============================] - 36s 165ms/step - loss: 0.0303 - accuracy: 0.9901 - val_loss: 0.0303 - val_

In [11]:
model.save('smnist.h5')

c:\Users\Yuval\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
